<a href="https://colab.research.google.com/github/Nell87/drivendata_richter/blob/main/script/feature_selection_and_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing data** 

In [22]:
####    INCLUDES  _______________________________________ #### 
#Loading Libraries:# 
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

####    READING TRAIN AND TEST DATA _______________________________________ #### 
train_values = data = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_values.csv")
train_labels = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_labels.csv")
test_values = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/test_values.csv")
train_merge = train_values.merge(train_labels, on = 'building_id', how = 'inner',)
print(train_merge.shape)

(260601, 40)


# **Feature engineering** 

## **Feature engineering*: Location features**
The features **geo_level_1_id, geo_level_2_id, geo_level_3_id** represent the geographic region in which building exists, from largest (level 1) to most specific sub-region (level 3). Possible values: level 1: 0-30, level 2: 0-1427, level 3: 0-12567.

For every location feature there is a high number of categorical values, so we'll apply feature engineering on them. We'll replace every value with their conditional probabilities respect to every damage_grade category

In [ ]:
# Function to replace a categorical feature with many values, with their conditional probabilities respecto to the predicted feature
def categoricalvalues_condprob(data, index, columns, new_column_name):
  # Create prob table
  probs = data.groupby(index).size().div(len(data))
  probs_group = data.groupby([index, columns]).size().div(len(data)).div(probs, axis=0, level=index).reset_index()
  probs_group.columns= [index, columns, new_column_name]
  probs_group_wide = probs_group.pivot(index=[index], columns = columns,values = new_column_name) #Reshape from long to wide
  probs_group_wide = probs_group_wide.reset_index()
 
 # Rename columns
  unique_values = np.unique(data.columns)
  unique_values = len(unique_values) 
  for i in range(unique_values):
    probs_group_wide.rename(columns={probs_group_wide.columns[-1]: 'test'}, inplace = True)



  # Add column to main dataset
  data_merge = data.merge(probs_group_wide, on=index, how='left')

  # Get rid of the categorical feature
  data_merge = data_merge.drop(index, axis=1)

  # Return dataset
  return probs_group_wide

# Apply the function
train_merge2 = categoricalvalues_condprob(train_merge, 'geo_level_1_id', 'damage_grade', 'prob_cond_geo_level_1')
#train_merge = categoricalvalues_condprob(train_merge, 'geo_level_2_id', 'damage_grade', 'prob_cond_geo_level_2')
#train_merge = categoricalvalues_condprob(train_merge, 'geo_level_3_id', 'damage_grade', 'prob_cond_geo_level_3')

train_merge2

# **Feature selection** 

In [61]:
unique_values = np.unique(train_merge.damage_grade)
unique_values = len(unique_values) 
range(unique_values)

range(0, 3)